## Reactions Parameters

Functions called upon to compute the reaction parameters of a system based on their input parameters

Last updated May 8, 2024 by Madeline Murphy

In [1]:
function RxnParametersArray(BEa, gamma, delta, alpha, beta)
 # Constants
        T = 298.15             # temperature [K]
        kB = 8.61733034e-5        # Boltzmann constant [eV/K]
        h = 4.1357e-15            # Planck constant [eV-s]
        ev = 1.602e-19            # Converts Joules to Ev
         
        # Unpacking Parameters
        gamma_ab = gamma[1]
        gamma_ac = gamma[2]
        
        delta_ab = delta[1]
        delta_ac = delta[2]
        
        alpha_a = alpha[1]
        alpha_b = alpha[2]
        alpha_c = alpha[3]
        
        beta_a = beta[1]
        beta_b = beta[2]
        beta_c = beta[3]
        
        ## Binding Energies from linear scaling relationships
        BEb = gamma_ab*BEa + (1-gamma_ab)*delta_ab
        BEc = gamma_ac*BEa + (1-gamma_ac)*delta_ac
    
        ## Surface Reactions 
        deltaG = Vector{Float64}(undef,3)
        deltaG[1] = - BEb - (-BEa)
        deltaG[2] = - BEc - (-BEb)
        deltaG[3] = - BEa - (-BEc)
        
        ## Activation Energies from BEP relationships
        Ea = Vector{Float64}(undef,3)
        Ea[1] = alpha_a*deltaG[1] + beta_a
        Ea[2] = alpha_b*deltaG[2] + beta_b
        Ea[3] = alpha_c*deltaG[3] + beta_c 
        Ea = max.(Ea,0)
  
        # Forward rate constant via transition state theory (TST)
        kf = (kB*T/h)*exp.(-Ea/(kB*T))
        
        # Equilibrium constant
        K = exp.(-deltaG/(kB*T))
        
        # Reverse rate constant
        kr = kf./K
    
        BE = hcat(BEa, BEb, BEc)
    
    return kf, kr, BE, Ea
end 

RxnParametersArray (generic function with 1 method)

In [ ]:
function RxnTimeConstants(BEa, gamma, delta, alpha, beta)
 # Constants
        T = 298.15             # temperature [K]
        kB = 8.61733034e-5        # Boltzmann constant [eV/K]
        h = 4.1357e-15            # Planck constant [eV-s]
        ev = 1.602e-19            # Converts Joules to Ev
         
        # Unpacking Parameters
        gamma_ab = gamma[1]
        gamma_ac = gamma[2]
        
        delta_ab = delta[1]
        delta_ac = delta[2]
        
        alpha_a = alpha[1]
        alpha_b = alpha[2]
        alpha_c = alpha[3]
        
        beta_a = beta[1]
        beta_b = beta[2]
        beta_c = beta[3]
        
        ## Binding Energies from linear scaling relationships
        BEb = gamma_ab*BEa + (1-gamma_ab)*delta_ab
        BEc = gamma_ac*BEa + (1-gamma_ac)*delta_ac
    
        ## Surface Reactions 
        deltaG = Vector{Float64}(undef,3)
        deltaG[1] = - BEb - (-BEa)
        deltaG[2] = - BEc - (-BEb)
        deltaG[3] = - BEa - (-BEc)
        
        ## Activation Energies from BEP relationships
        Ea = Vector{Float64}(undef,3)
        Ea[1] = alpha_a*deltaG[1] + beta_a
        Ea[2] = alpha_b*deltaG[2] + beta_b
        Ea[3] = alpha_c*deltaG[3] + beta_c 
        Ea = max.(Ea,0)
  
        # Forward rate constant via transition state theory (TST)
        kf = (kB*T/h)*exp.(-Ea/(kB*T))
        
        # Equilibrium constant
        K = exp.(-deltaG/(kB*T))
        
        # Reverse rate constant
        kr = kf./K
    
        BE = hcat(BEa, BEb, BEc)
    
    return kf, kr, BE
end 

In [4]:
function alpha_matrix(gamma, alpha, beta)
     # Constants
        T = 298.15             # temperature [K]
        kB = 8.61733034e-5        # Boltzmann constant [eV/K]
        h = 4.1357e-15            # Planck constant [eV-s]
        ev = 1.602e-19            # Converts Joules to Ev
         
        # Unpacking Parameters
        gamma_ab = gamma[1]
        gamma_ac = gamma[2]

        
        alpha_a = alpha[1]
        alpha_b = alpha[2]
        alpha_c = alpha[3]
        
        beta_a = beta[1]
        beta_b = beta[2]
        beta_c = beta[3]
    
    alpha_Array = Matrix{Float64}(undef,3,3)
    for i in range(1,3)
        alpha_Array[i,i] = 0.0;
    end
    
    # Reaction 1 A<-->B
    alpha_Array[1,2] = (alpha_a*(1-gamma_ab))^2
    alpha_Array[2,1] = ((1-alpha_a)*(1-gamma_ab))^2
    
    # Reaction 2 B<-->C
    alpha_Array[2,3] = (alpha_b*(gamma_ab-gamma_ac))^2
    alpha_Array[3,2] = ((1-alpha_b)*(gamma_ab-gamma_ac))^2
    
    # Reaction 3 C<-->A
    alpha_Array[3,1] = (alpha_c*(gamma_ac-1))^2
    alpha_Array[1,3] = ((1-alpha_c)*(gamma_ac-1))^2
    
    return alpha_Array
end

alpha_matrix (generic function with 1 method)

In [2]:
function Objective_Function(alpha_Array)
    C = 0.0;
    
    for i in range(1,3)
        for k in range(1,3) 
            if i <= k
                C = C + (alpha_Array[i,k])^2
            else 
                C = C - (alpha_Array[i,k])^2
            end
        end
    end
    return C
end

Objective_Function (generic function with 1 method)